# Make prediction using XGBoost models <br> on the Jurkat cell line data

In [ ]:
import os
import pandas as pd
import numpy as np
import joblib
from xgboost import XGBClassifier

In [ ]:
# Load the dataset
df = pd.read_csv("data/jurkat_beta_score.csv")
df = df.T
df.columns = df.iloc[0,:]
df.drop(["Gene"],inplace=True)
df

In [ ]:
# Get the features of the model  
features = pd.read_csv("models/xgboost_druggable/features.csv")
features = features.gene_id.tolist()
len(features)

In [ ]:
df_subset = df[features]
df_subset

In [ ]:
df_subset.to_csv("data/jurkat_prediction_data.csv")

In [ ]:
df_test = pd.read_csv("data/jurkat_prediction_data.csv")
df_test.drop(["Unnamed: 0"], axis=1, inplace=True)
df_test

In [ ]:
drug_list = pd.read_csv("models/xgboost_druggable/drug_list.txt", header=None)
drug_list = drug_list.iloc[:, 0].tolist()
len(drug_list)

In [ ]:
report_combined = pd.DataFrame()

for drug in drug_list:
    xgbc = joblib.load("models/xgboost_druggable/joblib/XGBoost_%s.joblib" % drug)
    y_pred = xgbc.predict(df_test)
    y_prob = xgbc.predict_proba(df_test)[:, 1]
    df1 = pd.DataFrame({'Drug':drug, 'Predicted':y_pred, 'Probability[1]':y_prob})
    report_combined=report_combined.append(df1)

In [ ]:
report_combined.to_csv("results/jurkat_prediction_results.csv")